In [ ]:
import numpy as np
from tqdm import tqdm
from time import time

import torchvision
from torchvision import models, transforms

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

In [ ]:
def accuracy(yhat,y):
    # si y encode les indexes
    if len(y.shape)==1 or y.size(1)==1:
        return (torch.argmax(yhat,1).view(y.size(0),-1)== y.view(-1,1)).double().mean()
    # si y est encodé en onehot
    return (torch.argmax(yhat,1).view(-1) == torch.argmax(y,1).view(-1)).double().mean()

def train(model,epochs,train_loader,test_loader,feature_extract=False):
    model = model.to(device)
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    
    params_to_update = model.parameters()
    print("params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    optim = torch.optim.Adam(params_to_update,lr=1e-3)
    
    print(f"running {model.name}")
    loss = nn.CrossEntropyLoss()
    for epoch in tqdm(range(epochs)):
        cumloss, cumacc, count = 0, 0, 0
        model.train()
        for x,y in train_loader:
            optim.zero_grad()
            x,y = x.to(device), y.to(device)
            yhat = model(x)
            l = loss(yhat,y)
            l.backward()
            optim.step()
            cumloss += l*len(x)
            cumacc += accuracy(yhat,y)*len(x)
            count += len(x)
        writer.add_scalar('loss/train',cumloss/count,epoch)
        writer.add_scalar('accuracy/train',cumacc/count,epoch)
        if epoch % 1 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, cumacc, count = 0, 0, 0
                for x,y in test_loader:
                    x,y = x.to(device), y.to(device)
                    yhat = model(x)
                    cumloss += loss(yhat,y)*len(x)
                    cumacc += accuracy(yhat,y)*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)
                writer.add_scalar('accuracy/test',cumacc/count,epoch)

def set_parameter_requires_grad(model, feature_extract):
    if feature_extract:
        for name,p in model.named_parameters():
            if "fc" not in name:
                p.requires_grad = False    
            else:
                p.requires_grad = True  
                
def get_test_data(dataloader, size):
    X_test, Y_test = next(iter(dataloader))
    batch_size = len(X_test)
    n = size//batch_size
    for i, batch in enumerate(dataloader):
        if i < n:
            X_tmp, Y_tmp = batch
            X_test = torch.cat((X_test, X_tmp), 0)
            Y_test = torch.cat((Y_test, Y_tmp), 0)
    return X_test, Y_test

In [ ]:
TB_PATH = "/tmp/logs/sceance2"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

resnet = models.resnet18(pretrained=True)

resnet.fc = nn.Linear(512, 10)

print(resnet.eval())

set_parameter_requires_grad(resnet, True)

In [ ]:
input_size = 224
batch_size = 128

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transformresnetTrain=transforms.Compose([ # Cette fois on utilise pas de grayscale car nous avons un gros modele pré-entrainé
        transforms.RandomResizedCrop(input_size), # selection aléatoire d'une zone de la taille voulue (augmentation des données en apprentissage)
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
transformresnetTest=transforms.Compose([
        transforms.Resize(input_size), # selection de la zone centrale de la taille voulue
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

resnet_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transformresnetTrain)
resnet_trainloader = torch.utils.data.DataLoader(resnet_trainset, batch_size=batch_size, pin_memory=True, shuffle=True)

resnet_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transformresnetTest)
resnet_testloader = torch.utils.data.DataLoader(resnet_testset, batch_size=batch_size, pin_memory=True, shuffle=True)

In [ ]:
## Entraînement du réseau
resnet.name = "resnet"
train(resnet, 1, resnet_trainloader, resnet_testloader)

In [ ]:
## Accuracy
X_test, Y_test = get_test_data(resnet_testloader, 1000) 
X_test, Y_test = X_test.to(device), Y_test.to(device)
print("Acc for resnet transfer learning :", accuracy(resnet(X_test), Y_test))

In [ ]:
for t in (20,40,60,80,100,120):
    t0 = time()
    resnet(X_test[:t])
    print("FPS:", t, " --> seconds:", (time() - t0))

In [ ]:
import os
PATH = "./"
torch.save(resnet.state_dict(), os.path.join(PATH,"resnet.pth"))

In [ ]:
PATH = "./"
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 10)
model.load_state_dict(torch.load(os.path.join(PATH,"vgg.pth"),map_location='cpu'))

model.eval() 
dummy_input = torch.randn(batch_size, 3, input_size, input_size)  
torch.onnx.export(model,   
                  dummy_input, 
                  "vgg.onnx",
                  export_params=True,
                  do_constant_folding=True, 
                  input_names = ['modelInput'],
                  output_names = ['modelOutput'])
print(" ") 
print('Model has been converted to ONNX') 